In [121]:
import requests
import json
import numpy as np
import pandas as pd
import os
from collections import Counter
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

format_spacing = {
    'ATOM' : { 
        'spacing': [6, 11, 16, 17, 20, 22, 26, 27, 38, 46, 54, 60, 66, 78, 80],
        'label': ['record_name', 'serial_number', 
                  'atom_name', 'alt_loc', 'res_name', 
                  'chain_id', 'res_seq', 'iCode', 
                  'x', 'y', 'z', 'occupancy', 'tempFactor', 
                  'element', 'charge']
    },
    'HELIX' : {
        'spacing': [6, 10, 14, 18, 20, 25, 26, 30, 32, 37, 38, 40, 70, 76],
        'label': ['record_name', 'serial_number', 'helix_id', 'init_res_name',
                  'init_chain_id', 'init_seq_num', 'init_iCode', 'end_res_name',
                  'end_chain_id', 'end_seq_num', 'end_iCode', 'helix_class',
                  'comment', 'length']
    },
    'SHEET' : {
        'spacing': [6, 10, 14, 16, 20, 22, 26, 27, 31, 33, 37, 38, 40, 45, 48, 50, 54, 55, 60, 63, 65, 69, 70],
        'label': ['record_name', 'strand', 'sheet_id', 'num_strands', 'init_res_name',
                  'init_chain_id', 'init_seq_num', 'init_iCode', 'end_res_name',
                  'end_chain_id', 'end_seq_num', 'end_iCode', 'sense', 'cur_atom', 'cur_res_name',
                  'cur_chain_id', 'cur_res_seq', 'cur_iCode', 'prev_atom', 'prev_res_name', 
                  'prev_chain_id', 'prev_res_seq', 'rev_iCode']
    }
}

atom_backchain = ['N', 'CA', 'C', 'O']


def parse_list_pdb( file_name: str):
    with open(file_name, 'r') as file_list_pdb:
        df_list_pdb = pd.read_csv(file_list_pdb, delimiter=' ', skipinitialspace=True)
    print(df_list_pdb)   
    return df_list_pdb

def parse_pdb_data( str_pdb: str, keyword: str, protein_name: str):
    if keyword in format_spacing.keys() and str_pdb.split(' ')[0] == keyword:
        l_label = format_spacing[keyword]['label']
        l_spacing = format_spacing[keyword]['spacing']
        if len(l_label) != len(l_spacing):
            raise Exception('length of label and spacing for {} not matching'.format(keyword))
        else:
            tmp_dict = dict()
            for i in range(len(l_spacing)):
                if i == 0:
                    tmp_dict[l_label[i]] = str_pdb[0:l_spacing[i]].strip()
                else:
                    tmp_dict[l_label[i]] = str_pdb[l_spacing[i - 1]:l_spacing[i]].strip()

            tmp_dict['protein_name'] = protein_name
            return tmp_dict

def process_pdb(pdb_name, len_atom, df_sheet):
    protein_name = pdb_name[:-1]
    protein_chain = pdb_name[-1]
    num_atoms_detected = 0

    if not (os.path.exists(pdb_dir + '/{}.pdb'.format(protein_name))):
        with open(pdb_dir + '/{}.pdb'.format(protein_name), 'wb') as f:
            print('Beginning {} pdb file download with requests'.format(pdb_name))
            session = requests.Session()
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            session.mount('https://', adapter)
            url = 'https://files.rcsb.org/view/{}.pdb'.format(protein_name)
            r = session.get(url)
            f.write(r.content)

    with open(pdb_dir + '/{}.pdb'.format(protein_name), 'r') as f:
        for line in f.readlines():
            dict_parsed_atom = parse_pdb_data(line, 'ATOM', protein_name)
            dict_parsed_helix = parse_pdb_data(line, 'HELIX', protein_name)
            dict_parsed_sheet = parse_pdb_data(line, 'SHEET', protein_name)
            
            if dict_parsed_sheet:
                if dict_parsed_sheet['cur_chain_id'] == protein_chain:
                    df_sheet = df_sheet.append(dict_parsed_sheet, ignore_index=True)
    df_csv = df_sheet.to_csv(r'C:\Users\g.m.de.la.torre\Documents\School Python\df_export.csv', index = None, header=True)
    
    return df_sheet


def pdb_parser(index_to_break, df_sheet):
    if index_to_break == -1:
        print('parsing all pdb in {}'.format(filename_list_pdb))
    else:
        print('parsing first {} proteins in {}'.format(index_to_break, filename_list_pdb))
    
    for index, pdb in df_pdb_list.iterrows():
        if index == index_to_break:
            break
        pdb_id = pdb['IDs']
        #print('parsing {}...'.format(pdb_id))
        df_sheet = process_pdb(pdb_id, pdb['length'], df_sheet)
    
    print("-"*120,"\nCurrent:",Counter(df_sheet.cur_res_name))
    print("\nDirection:",Counter(df_sheet.sense))
    print("\nInitial:",Counter(df_sheet.init_res_name))
    print("\nEnd",Counter(df_sheet.end_res_name))
    print("-"*120,"\n",df_sheet.info())
    return df_sheet

df_atom = pd.DataFrame()
df_sheet = pd.DataFrame()
df_helix = pd.DataFrame()
filename_list_pdb = 'cullpdb_pc30_res3.0_R1.0_d191017_chains18877.gz'
df_pdb_list = parse_list_pdb(filename_list_pdb)
pdb_dir = './pdb_data'

df_sheet = pd.read_csv('df_sheet_export.csv')
#ran to make the original df_sheet_export, after created you can just read the csv
#df_sheet = pdb_parser(-1, df_sheet)
print(df_sheet)

         IDs  length Exptl.  resolution  R-factor  FreeRvalue
0      12ASA     330   XRAY        2.20      0.16        0.29
1      16VPA     366   XRAY        2.10      0.19        0.26
2      19HCA     292   XRAY        1.80      0.17        1.00
3      1A0AA      63   XRAY        2.80      0.23        0.28
4      1A0IA     348   XRAY        2.60      0.22        0.34
...      ...     ...    ...         ...       ...         ...
18872  6ULOA     334   XRAY        1.30      0.14        0.17
18873  7A3HA     303   XRAY        0.95      0.11        0.13
18874  7AHLA     293   XRAY        1.89      0.20        0.26
18875  7ODCA     424   XRAY        1.60      0.20        0.23
18876  8ABPA     306   XRAY        1.49      0.17        1.00

[18877 rows x 6 columns]
       cur_atom cur_chain_id cur_iCode cur_res_name  cur_res_seq end_chain_id  \
0             N            A       NaN          GLY           91            A   
1             N            A       NaN          GLU          120    

In [122]:
strands_df = df_sheet.drop_duplicates(subset=["protein_name", "sheet_id"], keep="last")
df_csv = strands_df.to_csv(r'C:\Users\g.m.de.la.torre\Documents\School Python\df_strands.csv', index = None, header=True)

5. Analyze the amino acids that prefer to form/be part of beta-sheets. 


In [123]:
print("-"*120,"\nCurrent:",Counter(df_sheet.cur_res_name))
print("\nDirection:",Counter(df_sheet.sense))
print("\nInitial:",Counter(df_sheet.init_res_name))
print("\nEnd:",Counter(df_sheet.end_res_name))

------------------------------------------------------------------------------------------------------------------------ 
Current: Counter({'VAL': 20473, 'LEU': 17167, 'ILE': 16399, 'PHE': 9150, 'ALA': 8617, 'TYR': 8380, 'THR': 7962, 'ARG': 6404, 'LYS': 6337, 'SER': 6187, 'GLY': 6177, 'GLU': 6026, 'GLN': 4060, 'ASP': 3296, 'HIS': 3263, 'ASN': 2936, 'TRP': 2736, 'MET': 2666, 'CYS': 2192, 'MSE': 775, 'PRO': 168, 'MLY': 18, 'KCX': 10, 'UNK': 6, 'CME': 2, 'KPI': 2, 'CSO': 2, 'SEC': 1, 'LVN': 1, 'SEP': 1, 'CSD': 1, 'GPL': 1, 'PTR': 1, '6V1': 1, 'OCY': 1, 'MLZ': 1, 'SMC': 1})

Direction: Counter({-1: 97784, 1: 43628, 0: 3, 9: 2, 8: 1, 7: 1, 28: 1, 16: 1})

Initial: Counter({'VAL': 16965, 'LEU': 12828, 'ILE': 12054, 'THR': 10851, 'LYS': 9409, 'GLY': 8872, 'ALA': 8441, 'ARG': 8382, 'PHE': 7404, 'GLU': 7258, 'SER': 7106, 'TYR': 6975, 'GLN': 4983, 'HIS': 3852, 'ASN': 3527, 'ASP': 3172, 'TRP': 2588, 'MET': 2294, 'CYS': 2160, 'PRO': 1495, 'MSE': 747, 'MLY': 27, 'CME': 5, 'TYI': 4, 'UNK': 4, 'OCS':

5. Analyze the number of strands in beta-sheets.

In [124]:
print("\nStrands:",Counter(strands_df.num_strands))


Strands: Counter({2: 11143, 4: 8708, 3: 5883, 5: 5820, 6: 4187, 7: 2978, 8: 1904, 9: 1070, 10: 624, 12: 247, 11: 195, 14: 118, 13: 102, 16: 52, 18: 33, 15: 30, 17: 24, 20: 22, 19: 18, 1: 10, 22: 8, 21: 7, 23: 7, 28: 6, 25: 5, 36: 4, 24: 4, 31: 2, 26: 2, 32: 2, 43: 1, 27: 1, 30: 1, 34: 1, 48: 1, 75: 1})


7. Analyze the length of bonds and the angle between different type of backbone atoms in
general and for individual AAs.

In [125]:
df_atom = pd.read_csv('df_atom_export.csv')
protein_list = df_atom.drop_duplicates('protein_name')
print(protein_list['protein_name'])

0         12AS
2559      16VP
5016      19HC
7222      1A0A
7720      1A0I
          ... 
392151    1DP7
392775    1DPJ
393008    1DQ3
396763    1DQG
397848    1DQP
Name: protein_name, Length: 220, dtype: object


In [126]:
import math
import matplotlib.pyplot, pylab
import statistics

distance_list = []
def distance(x1, y1, z1, x2, y2, z2):  
       
    d = math.sqrt(math.pow(x2 - x1, 2) +
                math.pow(y2 - y1, 2) +
                math.pow(z2 - z1, 2)* 1.0) 
    distance_list.append(d)
    
def dis (protein):
    for index in protein.index:
        try:
            x1 = protein['x'][index]
            y1 = protein['y'][index]
            z1 = protein['z'][index]
            x2 = protein['x'][index+1]
            y2 = protein['y'][index+1]
            z2 = protein['z'][index+1]
            distance(x1, y1, z1, x2, y2, z2) 
        except:
            continue

for name in protein_list['protein_name']:
    df_pro_name = (df_atom[df_atom['protein_name'] == name])
    df_pro_name = (df_pro_name[df_pro_name['atom_name'] == 'CA']).reset_index(drop=True)
    dis(df_pro_name)
print('Distance from CA to CA')
print('Max:',max(distance_list))
print('Min:',min(distance_list))
print('Mean:',statistics.mean(distance_list))
print('Median:',statistics.median(distance_list))


Distance from CA to CA
Max: 113.35855197116803
Min: 0.0064807406984157805
Mean: 3.826283882257711
Median: 3.8045274345179854
